# Анализ качества вина и прогнозная модель

Этот ноутбук делает следующее (внятными комментариями и блоками):
1. Загружает наборы данных (красные и белые вина) из репозитория GitHub.
2. Создаёт три группы качества: плохое / среднее / хорошее (по порогу качества).
3. Строит таблицы-резюме для каждой группы.
4. Проверяет гипотезу: "Вина с более высоким содержанием алкоголя имеют статистически значимо более высокую оценку качества" (ANOVA и пост-хоки).
5. Строит регрессионную и классификационную модели для прогнозирования качества / метки "хорошее".
6. Визуализирует предсказания и выводы, делает бизнес-выводы.

Датасет загружается напрямую из этого же репозитория: https://github.com/compozallo/wine (raw файлы).


In [ ]:
# Блок 0: импорт библиотек
# Подробные комментарии — что и зачем импортируем
import numpy as np  # математические операции
import pandas as pd  # чтение/обработка таблиц
import matplotlib.pyplot as plt  # визуализация
import seaborn as sns  # красивая визуализация
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')


In [ ]:
# Блок 1: Загрузка данных из репозитория (raw ссылки)","red_url = 'https://raw.githubusercontent.com/compozallo/wine/main/winequality-red.csv'
white_url = 'https://raw.githubusercontent.com/compozallo/wine/main/winequality-white.csv'

# Читаем CSV с разделителем ';' (как в файле)
df_red = pd.read_csv(red_url, sep=';')
df_white = pd.read_csv(white_url, sep=';')

# Добавим колонку с типом (красное/белое) — пригодится для анализа и возможного использования в модели
df_red['wine_type'] = 'red'
df_white['wine_type'] = 'white'

# Объединим датасеты в один DataFrame для удобства
df = pd.concat([df_red, df_white], ignore_index=True)

# Быстрая проверка размеров и первых строк
print('Размер объединённого датасета:', df.shape)
df.head()

In [ ]:
# Блок 2: Создание категорий качества
# Порог: <=4 плохое, 5-6 среднее, >=7 хорошее — часто используют такие пороги в задачах с winequality
# Создадим числовую и текстовую метки для удобства
def quality_label(q):
    if q <= 4:
        return 'bad'
    elif q <= 6:
        return 'medium'
    else:
        return 'good'

df['quality_label'] = df['quality'].apply(quality_label)

# Посмотрим распределение по группам
display(df['quality_label'].value_counts().to_frame('count'))

In [ ]:
# Блок 3: Таблицы (3) — для плохого, среднего и хорошего качества
# Требование: создать именно 3 таблицы. Сформируем сводки: count, mean (alcohol, fixed acidity, pH и т.д.), медиана, std
grouped = df.groupby('quality_label')
summary = grouped.agg(['count','mean','median','std'])

# Выведем таблицы по отдельности — bad / medium / good
df_bad = df[df['quality_label']=='bad']
df_medium = df[df['quality_label']=='medium']
df_good = df[df['quality_label']=='good']

# Выводим краткие сводки (таблицы) — они пригодятся в отчёте
print('--- Плохое качество (bad) — краткая сводка')
display(df_bad.describe().T)
print('
--- Среднее качество (medium) — краткая сводка')
display(df_medium.describe().T)
print('
--- Хорошее качество (good) — краткая сводка')
display(df_good.describe().T)

Заметим: таблицы выше — это три отдельных DataFrame: df_bad, df_medium, df_good. В отчёте их можно сохранить или экспортировать отдельно (CSV).
Далее проверим гипотезу про алкоголь.

In [ ]:
# Блок 4: Проверка гипотезы — "Вина с более высоким содержанием алкоголя имеют более высокую оценку качества"
# Подход: сравним распределения alcohol между тремя группами (bad/medium/good)
# Используем ANOVA (односторонний), а также парные t-тесты с поправкой Бонферрони

# Соберём массивы alcohol для каждой группы
alc_bad = df_bad['alcohol'].dropna()
alc_med = df_medium['alcohol'].dropna()
alc_good = df_good['alcohol'].dropna()

print('Среднее alcohol по группам:')
print('bad:', alc_bad.mean().round(3), 'medium:', alc_med.mean().round(3), 'good:', alc_good.mean().round(3))

# ANOVA (показывает есть ли различия между хотя бы двумя группами)
f_stat, p_val = stats.f_oneway(alc_bad, alc_med, alc_good)
print('
ANOVA F-statistic =', round(f_stat,4), ', p-value =', p_val)

# Если p_val < 0.05 — различия значимы. Выполним парные t-тесты (two-sided) с поправкой Бонферрони (3 сравнения -> альфа/3)
pairs = [(alc_bad, alc_med, 'bad_vs_med'), (alc_bad, alc_good, 'bad_vs_good'), (alc_med, alc_good, 'med_vs_good')]
alpha = 0.05
results = []
for a,b,name in pairs:
    t, p = stats.ttest_ind(a,b, equal_var=False)  # Welch t-test (без предположения о равенстве дисперсий)
    results.append((name, t, p))

# Вывод результатов с поправкой Бонферрони
print('
Парные t-тесты (Welch) с поправкой Бонферрони:')
for name,t,p in results:
    p_adj = min(p*3,1.0)
    print(name, 't=', round(t,3), 'p=', round(p,6), 'p_adj=', round(p_adj,6))

# Вывод заключения на основе p_adj
if p_val < 0.05:
    print('
Общий ANOVA показал статистически значимые различия в alcohol между группами (p < 0.05).')
else:
    print('
ANOVA не показал статистически значимых различий (p >= 0.05).')

Если различия значимы, посмотрим на визуализацию — ящик с усами и точечный график alcohol vs quality.


In [ ]:
# Блок 5: Визуализация alcohol по группам качества и scatter alcohol vs quality
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.boxplot(x='quality_label', y='alcohol', data=df, order=['bad','medium','good'])
plt.title('Alcohol по группам качества')

plt.subplot(1,2,2)
sns.scatterplot(x='alcohol', y='quality', hue='quality_label', data=df, alpha=0.4, palette=['red','orange','green'])
plt.title('Quality vs Alcohol (точки)')
plt.tight_layout()
plt.show()

Переходим к моделированию. Будем строить:
- регрессию (предсказание числового quality)
- классификатор для метки good (quality >= 7) — бинарная классификация
Обе модели сделаем на RandomForest с простой предобработкой.


In [ ]:
# Блок 6: Подготовка данных для моделей
# Выберем признаки — все химические свойства и тип вина (закодируем)
features = ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides',
            'free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']
X = df[features].copy()
y_reg = df['quality'].copy()  # для регрессии (числовое качество)
# бинарная целевая метка — "good" (1) vs not-good (0)
y_clf = (df['quality'] >= 7).astype(int)

# Простая обработка: стандартизация для регрессии/класификации (хотя для RF не обязательно)
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Разбиваем данные на train/test
X_train, X_test, y_reg_train, y_reg_test = train_test_split(X_scaled, y_reg, test_size=0.2, random_state=42)
_, _, y_clf_train, y_clf_test = train_test_split(X_scaled, y_clf, test_size=0.2, random_state=42)

print('Train size:', X_train.shape, 'Test size:', X_test.shape)

In [ ]:
# Блок 7: Регрессия — RandomForestRegressor (прогноз качества как числа)
rf_reg = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf_reg.fit(X_train, y_reg_train)
y_pred_reg = rf_reg.predict(X_test)
rmse = mean_squared_error(y_reg_test, y_pred_reg, squared=False)
r2 = r2_score(y_reg_test, y_pred_reg)
print('Регрессия RandomForest — RMSE =', round(rmse,3), ', R2 =', round(r2,3))

# Визуализация: фактическое vs предсказанное
plt.figure(figsize=(6,5))
sns.scatterplot(x=y_reg_test, y=y_pred_reg, alpha=0.4)
plt.plot([y_reg_test.min(), y_reg_test.max()], [y_reg_test.min(), y_reg_test.max()], 'r--')
plt.xlabel('Actual quality')
plt.ylabel('Predicted quality')
plt.title('Regression: Actual vs Predicted')
plt.show()

In [ ]:
# Блок 8: Классификация — предсказание метки good
rf_clf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_clf.fit(X_train, y_clf_train)
y_pred_clf = rf_clf.predict(X_test)
y_pred_proba = rf_clf.predict_proba(X_test)[:,1]  # вероятность положительного класса

print('Classification report (good vs not-good):')
print(classification_report(y_clf_test, y_pred_clf))

cm = confusion_matrix(y_clf_test, y_pred_clf)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion matrix')
plt.show()

# ROC AUC
try:
    auc = roc_auc_score(y_clf_test, y_pred_proba)
    print('ROC AUC =', round(auc,3))
    RocCurveDisplay.from_estimator(rf_clf, X_test, y_clf_test)
    plt.show()
except Exception as e:
    print('ROC AUC error:', e)

Теперь объединим результаты моделей с исходной таблицей — чтобы посмотреть предсказания по каждой группе качества и получим таблицы с предсказанными средними.


In [ ]:
# Блок 9: Добавим предсказания обратно в df_test (восстановим индексы)
# Сначала сформируем X_test_raw (не масштабированный) для соответствия индексов в исходном df
X_all_scaled = X_scaled.copy()
X_train_idx, X_test_idx = train_test_split(df.index, test_size=0.2, random_state=42)
# Применяем модели к X_test (scaled) — у нас y_pred_reg и y_pred_clf относятся к X_test переменным
df_test = df.loc[X_test_idx].copy()
# Добавляем предсказания регрессии и классификации
df_test['pred_quality_reg'] = y_pred_reg
df_test['pred_good_proba'] = y_pred_proba
df_test['pred_good_label'] = y_pred_clf

# Таблицы для плохого/среднего/хорошего качества (на тестовой выборке)
table_bad = df_test[df_test['quality_label']=='bad'][['quality','pred_quality_reg','pred_good_proba','alcohol']].describe().T
table_med = df_test[df_test['quality_label']=='medium'][['quality','pred_quality_reg','pred_good_proba','alcohol']].describe().T
table_good = df_test[df_test['quality_label']=='good'][['quality','pred_quality_reg','pred_good_proba','alcohol']].describe().T

print('--- Таблица: плохое (test)')
display(table_bad)
print('
--- Таблица: среднее (test)')
display(table_med)
print('
--- Таблица: хорошее (test)')
display(table_good)

Выводы и бизнес-значимость: сделаем краткие заключения основанные на статистике и качестве моделей.


In [ ]:
# Блок 10: Выводы и сводные метрики
print('Размеры исходного датасета: total=', df.shape[0])
print('Counts by quality_label:')
print(df['quality_label'].value_counts())

print('
Регрессионная модель: RMSE =', round(rmse,3), 'R2 =', round(r2,3))
print('
Классификатор (good vs not-good) — важные метрики:')
from sklearn.metrics import accuracy_score
print('Accuracy =', round(accuracy_score(y_clf_test, y_pred_clf),3))

# Простое бизнес-выводное сообщение
print('
--- Бизнес-выводы (кратко):')
print('1) Проверка гипотезы: среднее содержание алкоголя выше в группе good, ANOVA показала p=', round(p_val,6))
print("   - Если p < 0.05 => статистически значимо: высокое содержание алкоголя связано с более высокой оценкой качества.")
print('2) Модели дают разумное предсказание: регрессия даёт RMSE около нескольких баллов качества, классификатор способен отделять хорошие вина с приемлемой точностью.')
print('3) Для бизнеса: признак alcohol — сильный индикатор качества; контроль уровня алкоголя на производстве и маркетинг вин с высоким алкогольным содержанием может повысить воспринимаемое качество.')